# Securing an Atoti session

Securing a session comes in two parts:
1. Implementing an authentication mechanism to secure access to the session
2. Restricting access of modules or data access by users based on the roles granted

__Security implementation is one of the [locked features](https://docs.atoti.io/latest/how_tos/unlock_all_features.html) that is not available in Atoti Community Edition. To try out this notebook, you can request for an [evaluation license](https://atoti.io/evaluation-license-request/) to unlock all the features.__ 

Atoti supports multiple [authentication mechanisms](https://docs.atoti.io/latest/lib/atoti/atoti.config.authentication.html) to cater to the needs of our end users.  

This notebook puts together 4 different ways we could authenticate users in Atoti, so that you can compare the differences in the implementation.  

You could alternatively focus on the individual implementation in the following notebooks:
- [01-Basic-authentication.ipynb](./01-Basic-authentication.ipynb)
- [02-OIDC-Auth0.ipynb](./02-OIDC-Auth0.ipynb)
- [03-OIDC-Google.ipynb](./03-OIDC-Google.ipynb)
- [04-LDAP.ipynb](./03-OIDC-Google.ipynb)

We will explore the Atoti security features using the [Top 50 Fast Food](https://www.kaggle.com/datasets/stetsondone/top50fastfood) dataset from Kaggle, combined with its parent company information sourced from the internet.  

__Note__:  

The notebook is structured in this order:
1. Authentication setup during session instantiation
2. Create BI analytics platform with Atoti
3. Users and roles management with Atoti

Mainly, we look at the roles last because we need to know the names of the tables and columns which we want to impose restrictions on.  
Thereafter, we can create the roles with restrictions.

<div style="text-align: center;" ><a href="https://www.atoti.io/?utm_source=gallery&utm_content=security-implementation" target="_blank" rel="noopener noreferrer"><img src="https://data.atoti.io/notebooks/banners/Discover+Atoti+now.jpg" alt="Try Atoti"></a></div>

In [1]:
import os
import requests
import atoti as tt
import ipywidgets as widgets
import pandas as pd

In [2]:
AUTH_BASIC = "BASIC"
AUTH_OIDC_GOOGLE = "OIDC - Google"
AUTH_OIDC_AUTH0 = "OIDC - Auth0"
AUTH_LDAP = "LDAP"

## 1. Authentication mechanism selection

Assuming you have the corresponding authentication provider configured with the users (and roles where applicable), select the mode of authentication you would like to test.  

__NOTE:__ Restart the session if the mode of authentication is changed. Session will be recreated and consequently, data has to be reloaded and the cube has to be recreated as well.

In [3]:
auth_mode = widgets.RadioButtons(
    options=[AUTH_BASIC, AUTH_OIDC_GOOGLE, AUTH_OIDC_AUTH0, AUTH_LDAP],
    value=AUTH_BASIC,
    description="Mode of authentication:",
    disabled=False,
    style={"description_width": "initial"},
)

display(auth_mode)

RadioButtons(description='Mode of authentication:', options=('BASIC', 'OIDC - Google', 'OIDC - Auth0', 'LDAP')…

### 1.1. Authentication setup in Atoti  

We demonstrate below how we configure the authentication parameter of [`atoti.Session`](https://docs.atoti.io/latest/lib/atoti/atoti.session.html#atoti.Session.__init__) for different authentication mechanisms.  

In [4]:
def get_auth_mode():
    print("Mode of auth:", auth_mode.value)

    if auth_mode.value == AUTH_BASIC:
        # https://docs.atoti.io/latest/lib/atoti/atoti.config.authentication.basic_authentication_config.html
        # The realm value (case-sensitive), in combination with the canonical root URL of the server being accessed, defines the protection space.
        authentication = tt.BasicAuthenticationConfig(realm="atoti Realm")

    elif auth_mode.value == AUTH_OIDC_AUTH0:
        # https://docs.atoti.io/latest/lib/atoti/atoti.config.authentication.oidc_config.html
        authentication = tt.OidcConfig(
            provider_id="auth0",
            issuer_url="https://dev-5m2svhd0.us.auth0.com/",
            client_id=os.environ["AUTH0_CLIENT_ID"],
            client_secret=os.environ["AUTH0_CLIENT_SECRET"],
            name_claim="name",
            scopes=["email", "profile", "username", "roles"],
            roles_claims=[
                "https://dev-5m2svhd0:us:auth0:com/roles",
            ],
        )

    elif auth_mode.value == AUTH_OIDC_GOOGLE:
        # https://docs.atoti.io/latest/lib/atoti/atoti.config.authentication.oidc_config.html
        authentication = tt.OidcConfig(
            provider_id="google",
            issuer_url="https://accounts.google.com",
            client_id=os.environ["GOOGLE_CLIENT_ID"],
            client_secret=os.environ["GOOGLE_CLIENT_SECRET"],
            scopes=["https://www.googleapis.com/auth/userinfo.email"],
            name_claim="email",
        )
    elif auth_mode.value == AUTH_LDAP:
        authentication = tt.LdapConfig(
            url="ldap://localhost:10389",
            base_dn="dc=example,dc=com",
            user_search_base="ou=people",
            group_search_base="ou=roles",
        )

    return authentication

Client ids and secrets in the case of OIDC should be kept private. As suggested in the [documentation](https://docs.atoti.io/latest/how_tos/secure_a_session.html#Configuring-the-authentication-mechanism), connection details can be read from environment variables for improved security.  

#### 1.1.1. [OIDC - Auth0 only] Authentication config mapping against Auth0 setup  

In Auth0, navigate to Actions > Library and click on "Build Custom" button to create a trigger for the `Login / Post Login` as follows (update namespace according to your environment):

```
exports.onExecutePostLogin = async (event, api) => {
  const namespace = 'https://dev-5m2svhd0.us.auth0.com';
  if (event.authorization) {
    api.idToken.setCustomClaim(`${namespace}/roles`, event.authorization.roles);
    api.accessToken.setCustomClaim(`${namespace}/roles`, event.authorization.roles);
  }
};
```

Thereafter, refer to the following for the setting mapping against Auth0:
<img src="https://data.atoti.io/notebooks/security/img/auth0-setup.png" width="80%" />  

`name_claim` could be email or name, depending on what you would like to see reflected in the application:  

<img src="https://data.atoti.io/notebooks/security/img/displayed_id.png" width="80%" />  

#### 1.1.2. [OIDC - Google only] Authentication config mapping against Google setup

Firstly, we have to configure the _OAuth consent screen_ and registered our app. Remember to set the "User type" to _Internal_, so that our app is limited to Google Workspace users within our organisation.  

<img src="https://data.atoti.io/notebooks/security/img/google-oauth-consent-screen.png" width="70%" />

Proceed to create a credential of type "OAuth client IDs" under __APIs & Services > Credentials__ in Google Cloud.  
Thereafter, refer to the following for the setting mapping against google:  
<img src="https://data.atoti.io/notebooks/security/img/google-setup.png" width="70%" />

#### 1.1.3. [LDAP] Authentication config mapping

In our example, we used the [Apache DS](https://directory.apache.org/apacheds/) to configure our LDAP server.

Use `user_search_base` to search the group where the user id (UID) resides under.  
Use `group_search_base` to search the group where the common names (CN) resides under.  

<img src="https://data.atoti.io/notebooks/security/img/ldap-setup.png" width="70%" />  

__All users should have the role equivalent to `ROLE_USER` to be able to access Atoti UI.__

### 1.2. Session instantiation with authentication 

For OIDC setup, it is important to fix the port to configure some URLs for callbacks.  

<img src="https://data.atoti.io/notebooks/security/img/callbacks.png" width="70%" />  

We use the [provider id](https://docs.atoti.io/latest/lib/atoti/atoti.config.authentication.oidc_config.html#atoti.OidcConfig.provider_id) (e.g. google or auth0) to build the redirect URL:

`f"{session_url}/login/oauth2/code/{provider_id}"`  

In any case, setting up firewall is part of the security setup and this would require a fixed port for the application too.

In [5]:
session = tt.Session(
    port=10018,
    authentication=get_auth_mode(),
    user_content_storage="./content",
    java_options=["-Dlogging.level.org.springframework.security=DEBUG"],
)

Mode of auth: BASIC


#### 1.2.1 Debug security setup

During the initial setup, it is useful to configure the [Spring Security logging](https://www.baeldung.com/spring-security-enable-logging) to help in debugging any potential issues in the connectivity.  
As shown in the above code snippet, we can turn on logging with `logging.level.org.springframework.security` set to the `DEBUG` level using the `java_options`.

## 2. Create BI analytics platform with Atoti

Once the session is created, we can proceed with the usual data loading into Atoti table, cube and measures creation.  
Remember to re-execute these cells if you have changed the mode of authentiction.

### 2.1 Table creation

Although we can [`create table`](https://docs.atoti.io/latest/lib/atoti/atoti.session.html#atoti.Session.create_table) before loading data in, we used `read_csv` in our example to create and load data into the Atoti tables.

In [6]:
base_tbl = session.read_csv(
    "s3://data.atoti.io/notebooks/security/data/parent_co.csv",
    table_name="parent_co",
    keys=["company", "parent_company"],
    process_quotes=True,
)
base_tbl.head()

,,name
parent_company,company,
Inspire Brands,arbys,Arby's
"Domino's Pizza, Inc.",dominos,Domino's
"Papa Murphy's Holdings, Inc.",papa_murphys,Papa Murphy's
Focus Brands,auntie_annes,Auntie Anne's
Panda Restaurant Group,panda_express,Panda Express


In [7]:
enrichment_tbl = session.read_csv(
    "s3://data.atoti.io/notebooks/security/data/top_50_fast_food_US.csv",
    table_name="top_50",
    keys=["company"],
)
enrichment_tbl.head()

,category,sales_in_millions_2019,sales_per_unit_thousands_2019,franchised_units_2019,company_owned_units_2019,total_units_2019,unit_change_from_2018
company,,,,,,,
popeyes_chicken,chicken,3750,1541,2458,41,2499,131
del_taco,global,850,1554,296,300,596,16
jack_in_the_box,burger,3505,1565,2106,137,2243,6
tim_hortons,sandwich,840,1165,715,0,715,-12
mcdonalds,burger,40413,2912,13154,692,13846,-66


In [8]:
base_tbl.join(enrichment_tbl)

### 2.2. Cube creation

In [9]:
cube = session.create_cube(base_tbl, "Fast food analysis")

### 2.3 Measures creation

In [10]:
h, l, m = cube.hierarchies, cube.levels, cube.measures

In [11]:
m["sales_in_millions"] = tt.agg.sum(enrichment_tbl["sales_in_millions_2019"])
m["sales_per_unit_thousands"] = tt.agg.sum(
    enrichment_tbl["sales_per_unit_thousands_2019"]
)
m["franchised_units"] = tt.agg.sum(enrichment_tbl["franchised_units_2019"])
m["company_owned_units"] = tt.agg.sum(enrichment_tbl["company_owned_units_2019"])
m["total_units"] = tt.agg.sum(enrichment_tbl["total_units_2019"])
m["unit_change_from_2018"] = tt.agg.sum(enrichment_tbl["unit_change_from_2018"])

In [12]:
m["% franchised"] = m["franchised_units"] / m["total_units"]
m["% franchised"].formatter = "DOUBLE[0.00%]"

## 3. Manage users and roles with Atoti

Let's create some constants for the user id of the users we will be creating.  

In [13]:
ADMIN_USER = "atoti_admin"
ATOTI_USER = "atoti_user"

INSPIRE_USER1 = "Inspire_user1"
INSPIRE_USER2 = "Inspire_user2"
INSPIRE_MANAGER = "Inspire_manager"
RESTAURANT_USER1 = "Restaurant_user1"
RESTAURANT_USER2 = "Restaurant_user2"
RESTAURANT_MANAGER = "Restaurant_manager"

### 3.1. [Basic Authentication only] User creation  

Users' information is retrieved from authentication provider if one is used. However, in the case of basic authentication, users are created in Atoti.
Making use of the [BasicSecurity](https://docs.atoti.io/latest/lib/atoti-query/atoti_query.security.basic_security.html#atoti_query.security.basic_security.BasicSecurity) module, we create an Atoti administrator and a generic Atoti user as shown below:

In [14]:
if auth_mode.value == AUTH_BASIC:
    session.security.basic.credentials[ADMIN_USER] = "password"
    session.security.basic.credentials[ATOTI_USER] = "password"

We cherry-picked two parent companies - Inspire Brands and Restaurant Brands International LLC to demonstrate the roles and access management.  
We will create the users based on the list available on [basic_user_pwd.csv](https://data.atoti.io/notebooks/security/users/basic_user_pwd.csv).

In [15]:
if auth_mode.value == AUTH_BASIC:
    users_df = pd.read_csv(
        "https://data.atoti.io/notebooks/security/users/basic_user_pwd.csv"
    )
    for row in users_df.to_dict(orient="records"):
        session.security.basic.credentials[row["User"]] = row["Password"]

        print(f"Create user: {row['User']}")

Create user: Inspire_user1
Create user: Inspire_user2
Create user: Inspire_manager
Create user: Restaurant_user1
Create user: Restaurant_user2
Create user: Restaurant_manager


The system reserved role `ROLE_USER` is automatically assigned to the created users. This meant that these users will all be able to access all the web application and all the available data.  
We can, however, restrict access for the users by assigning them roles with restricted access. 

### 3.2. Roles management in Atoti

#### 3.2.1. Atoti reserved roles  

The below roles are reserved in Atoti and should not be altered by users:
- ROLE_ADMIN: able to access all objects in the web application
- ROLE_USER: able to access all data by default. Access to objects such as dashboards, folders, widgets etc is only upon sharing access granted to role.

__All users, including the administrator, require the role *ROLE\_USER* to be able to access the Atoti UI.__

Let's assume both users, *atoti\_admin* and *atoti\_user*, have been granted the role __ROLE_USER__ and *atoti\_admin* is also granted the role __ROLE_ADMIN__.  
While both *atoti\_admin* and *atoti\_user* are able to access all data, *atoti\_admin* is able to access all objects such as folders and dashboards.  
*atoti\_user* is able to access only the objects created by the user him/herself. Objects created by other users can only be access upon shared access granted.  


In [16]:
if auth_mode.value == AUTH_BASIC:
    session.security.individual_roles[ADMIN_USER] = {"ROLE_USER", "ROLE_ADMIN"}
    session.security.individual_roles[ATOTI_USER] = {"ROLE_USER"}

Try logging in to the Atoti UI from the below link using either of the users `atoti_admin` or `atoti_user`.

In [17]:
session.link()

Open the notebook in JupyterLab with the Atoti extension enabled to see this link.

##### 3.2.1.1 Sharing role configuration 

In Atoti version before v0.8.0, we assign the role `ROLE_SHARE` to users in order to be able to let them share objects such as dashboards, folders, widgets and filters.  
Both *atoti\_admin* and *atoti\_user* will not be able to share objects (via the "Share" icon as shown below) unless granted the role __ROLE_SHARE__.  

<img src="img/share_function.png" width="50%"/>  

From version v0.8.0 onwards, the role `ROLE_SHARE` is removed. Instead, users with the role `ROLE_USER` will the ability to perform sharing by default.  

<img src="img/admin_share.png" width="50%"/>


We can configure the sharing permission from the Atoti Admin UI which is accessible from the link below. Login with a user that has the role `ROLE_ADMIN` assigned.

In [18]:
session.link(path="/admin")

Open the notebook in JupyterLab with the Atoti extension enabled to see this link.

In case we want to limit the users who has the ability to perform sharing, we can either update the `canShare` permission for `ROLE_USER` to `false` from the Admin UI; or we can use the rest service as shown below:

In [19]:
from requests.auth import HTTPBasicAuth

# Rest API call to set mode to dark and display dimension names above hierarchies
response = requests.put(
    f"http://localhost:{session.port}/activeviam/content/rest/v7/files?path=ui/user_roles/ROLE_USER/permissions",
    auth=HTTPBasicAuth("atoti_admin", "password"),
    json={
        "content": '{"canShare": false}',
        "owners": ["atoti_admin"],
        "readers": ["ROLE_USER"],
        "overwrite": True,
        "recursive": True,
    },
)

In the event we want to control the sharing rights using roles from Authentication providers, we can create the intended role in the Admin UI.  
For instance, suppose the role that is supposed to have sharing permission is call `ROLE_SHARE`, we can run the below request to create the role with `canShare` set to `true`.  

In [20]:
# Rest API call to set mode to dark and display dimension names above hierarchies
response = requests.put(
    f"http://localhost:{session.port}/activeviam/content/rest/v7/files?path=ui/user_roles/ROLE_SHARE/permissions",
    auth=HTTPBasicAuth("atoti_admin", "password"),
    json={
        "content": '{"canShare": true}',
        "owners": ["atoti_admin"],
        "readers": ["ROLE_USER"],
        "overwrite": True,
        "recursive": True,
    },
)

Below is what we see in the Admin server:  

<img src="img/role_share_permission.png" />  

Now, any users who are assigned the role `ROLE_SHARE` will have the ability to perform sharing.  
In the below snippet, we grant `atoti_admin` the additional role `ROLE_SHARE` using `|` to append its existing list of roles. Now, only the admin user is able to perform sharing.

In [21]:
if auth_mode.value == AUTH_BASIC:
    session.security.individual_roles[ADMIN_USER] |= {"ROLE_SHARE"}

#### 3.2.2. Role creation with restrictions  

Data restriction is based on users' requirement. In our use case, we assumed two groups of users with data access limited to those of their parent company:
- users belonging to parent company _Inspire Brands_
- users belonging to parent company _Restaurant Brands International Inc._

Therefore, we will create two roles to apply the restrictions based on the `parent_company` column from the `parent_co` table.  
We will define key that is a tuple, consisting of the name of the table and its column, along with the restricted values imposed on it. 

__NOTE:__  
- We can skip role creation if there are no restrictions imposed on the role. 
- The value provided under the restrictions is cap-sensitive.

In [22]:
ROLE_INSPIRE = "ATOTI_ROLE_INSPIRE"
ROLE_RESTAURANT = "ATOTI_ROLE_RESTAURANT"

session.security.restrictions.update(
    {
        ROLE_INSPIRE: (base_tbl["parent_company"] == "Inspire Brands"),
        ROLE_RESTAURANT: (
            base_tbl["parent_company"] == "Restaurant Brands International Inc."
        ),
    }
)

#### 3.2.3. Restricted access from combination of roles

Multiple roles can be assigned to the same user. To demonstrate how the access will change when this happens, we create some other roles that restrict data access by the restaurant category, i.e. column `category` from the table `top_50`.

In [23]:
ROLE_BURGER = "ATOTI_ROLE_BURGER"
ROLE_SANDWICH = "ATOTI_ROLE_SANDWICH"
ROLE_SNACK = "ATOTI_ROLE_SNACK"

session.security.restrictions.update(
    {
        ROLE_BURGER: enrichment_tbl["category"] == "burger",
        ROLE_SANDWICH: enrichment_tbl["category"] == "sandwich",
        ROLE_SNACK: enrichment_tbl["category"] == "snack",
    }
)

When combined with the restricted role on the `parent_company`, user's access will be further restricted to based on the restriction of the added role.  

For instance, users who are assigned the role __ATOTI_ROLE_BURGER__ will be able to access all the data under _burger_ category restaurants, regardless of the parent companies.  

However, when the same user is also granted the role __ATOTI_ROLE_INSPIRE__, then the user can only access data of restaurants under parent company _Inspire Brands_ that is of category _burger_. 

### 3.3. [Basic Authentication only] Assigning user roles to user (Good reference for roles setup in authentication providers)  

We can grant Atoti roles directly to users created in Atoti without having to perform role mappings like in OIDC.
But, we can easily demonstrate how roles can be assigned to the users to restrict data access. It can be easily inferred and applied on the authentication providers when granting roles.

__NOTE:__ We will have to grant all users the role `ROLE_USER` before they can access the Atoti application.

#### 3.3.1 Multiple roles assignment  

We grant the managers only access to the data available under their parent companies.  
These restrictions will be applied under the role __ATOTI_ROLE_INSPIRE__ and __ATOTI_ROLE_RESTAURANT__ respectively.  

Also, the managers will be granted __ROLE_SHARE__ for them to share the objects such as dashboards and widgets for which they are the owners of.

In [24]:
if auth_mode.value == AUTH_BASIC:
    session.security.individual_roles[INSPIRE_MANAGER] = {
        "ROLE_USER",
        "ROLE_SHARE",
        ROLE_INSPIRE,
    }
    session.security.individual_roles[RESTAURANT_MANAGER] = {
        "ROLE_USER",
        "ROLE_SHARE",
        ROLE_RESTAURANT,
    }

Each company has two users that have even more restricted access than the managers.  
User 1 of each company can only access data for restaurants of category _burgers_ with role __ATOTI_ROLE_BURGER__.  
User 2 of each company can only access data for restaurants of category _sandwich_ and _snack_ with roles __ATOTI_ROLE_SANDWICH__ and __ATOTI_ROLE_SNACK__.  

Combined with either the role __ATOTI_ROLE_INSPIRE__ or __ATOTI_ROLE_RESTAURANT__, they will only see the specific category of restaurants under their parent companies.

In [25]:
if auth_mode.value == AUTH_BASIC:
    session.security.individual_roles.update(
        {
            INSPIRE_USER1: {"ROLE_USER", ROLE_INSPIRE, ROLE_BURGER},
            INSPIRE_USER2: {"ROLE_USER", ROLE_INSPIRE, ROLE_SANDWICH, ROLE_SNACK},
            RESTAURANT_USER1: {"ROLE_USER", ROLE_RESTAURANT, ROLE_BURGER},
            RESTAURANT_USER2: {"ROLE_USER", ROLE_RESTAURANT, ROLE_SANDWICH, ROLE_SNACK},
        }
    )

### 3.4. [For OIDC and LDAP only] Role assignment in Atoti

The names in the authentication provider can be different from those in Atoti.    
Roles are assigned to users in the authentication provider and restrictions applied based on this map.  

While a role from the authentication provider can be mapped to multiple Atoti roles, it is easier to modify user's access by updating the roles in the authentication provider.  
It is a design consideration whether to have a one-to-one role map or to grant multiple Atoti roles to an authentication provider role.  

E.g. By **mapping `AUTH0_USER: [ROLE_USER, ROLE_SHARE]` in Atoti code** meant that all the users with __AUTH0_USER__ role assigned will be able access the web application and to do sharing.  
Alternatively, when we code map the **AUTH0 roles individually to each reserved role** as listed in the previous cell, individual users are granted the roles __AUTH0_USER__ and __AUTH0_ROLE_SHARE__ in Auth0 as shown below. Users will be able to modify the role assignment from Auth0 instead of performing code updates.

<img src="https://data.atoti.io/notebooks/security/img/auth0_roles_assignment.png" width="50%" />

#### 2.4.1. Assign default roles to authenticated users

Previously, we removed the `canShare` permission for `ROLE_USER`.  

To demonstrate how we can assign default roles to authenticated users, we can default the roles such as __ROLE_SHARE__ for all users who are logged in successfully.  
By default the role __ROLE_SHARE__, all users now can perform sharing.

Depending on the authentication mechanism applied, use either [`session.security.oidc`](https://docs.atoti.io/latest/lib/atoti-query/atoti_query.security.oidc_security.html#atoti_query.security.oidc_security.OidcSecurity.default_roles) or [`session.security.ldap`](https://docs.atoti.io/latest/lib/atoti-query/atoti_query.security.ldap_security.html#atoti_query.security.ldap_security.LdapSecurity.default_roles) module to perform the operations.

In [26]:
if auth_mode.value in [AUTH_OIDC_AUTH0, AUTH_OIDC_GOOGLE]:
    session.security.oidc.default_roles.update(["ROLE_SHARE"])
elif auth_mode.value == AUTH_LDAP:
    session.security.ldap.default_roles.update(["ROLE_SHARE"])

__BE CAUTION__ when granting __ROLE_USER__ to users by default as users with this role will be able to access the application and the available data. It's better to grant __ROLE_USER__ to the users individually instead.

#### 3.4.2. Map roles between Authentication Provider and Atoti  

The names in the authentication provider can be different from those in Atoti.    
Roles are assigned to users in the authentication provider and associated to the Atoti roles in the `role_mapping` below.

##### 3.4.2.1. Role mapping for OIDC (Auth0 example)

In [27]:
# if auth_mode.value == AUTH_OIDC_AUTH0:
print("Map roles from Auth0 to roles in Atoti")
session.security.oidc.role_mapping.update(
    {
        # authentication provider roles: [Atoti reserved roles]
        "AUTH0_ADMIN": ["ROLE_ADMIN"],
        "AUTH0_USER": ["ROLE_USER"],
        # authentication provider roles:  [user-defined roles]
        "AUTH0_ROLE_SHARE": ["ROLE_SHARE"],
        "AUTH0_ROLE_INSPIRE": [ROLE_INSPIRE],
        "AUTH0_ROLE_RESTAURANT": [ROLE_RESTAURANT],
        "AUTH0_ROLE_BURGER": [ROLE_BURGER],
        "AUTH0_ROLE_SANDWICH": [ROLE_SANDWICH],
        "AUTH0_ROLE_SNACK": [ROLE_SNACK],
    }
)

Map roles from Auth0 to roles in atoti


##### 3.4.2.2. Role mapping for OIDC (Google example)

__Atoti is only able to use Google for authentication.__  
Roles are assigned to the users in Atoti and they can be granted without having to restart the application.  

Instead of mapping the roles from Authentication Provider as in the case of Auth0, we use the user's email address to map to the assigned Atoti roles.

In [28]:
if auth_mode.value == AUTH_OIDC_GOOGLE:
    print("Map roles from Google to roles in Atoti")
    session.security.oidc.role_mapping.update(
        {
            # Google user id: [Atoti roles]
            "inspire_m@test.com": ["ROLE_USER", ROLE_INSPIRE],
            "inspire_user1@test.com": [
                "ROLE_USER",
                ROLE_INSPIRE,
                ROLE_BURGER,
            ],
            "inspire_user2@test.com": [
                "ROLE_USER",
                ROLE_INSPIRE,
                ROLE_SANDWICH,
                ROLE_SNACK,
            ],
            "restaurant_m@test.com": ["ROLE_USER", ROLE_RESTAURANT],
            "restaurant_user1@test.com": [
                "ROLE_USER",
                ROLE_RESTAURANT,
                ROLE_BURGER,
            ],
            "restaurant_user2@test.com": [
                "ROLE_USER",
                ROLE_RESTAURANT,
                ROLE_SANDWICH,
                ROLE_SNACK,
            ],
        }
    )

##### 3.4.2.3. Role mapping for LDAP

Similar to Auth0, we can map the roles downloaded from LDAP to the roles in Atoti.  

The names in the authentication provider can be different from those in Atoti.  
Roles are assigned to users in the authentication provider and associated to the Atoti roles in the `role_mapping` below.  

Be mindful that regardless of the name used in LDAP, we have to __uppercase the LDAP role name__ in the mapping.

In [29]:
if auth_mode.value == AUTH_LDAP:
    print("Map roles from LDAP to roles in Atoti")
    session.security.ldap.role_mapping.update(
        {
            # LDAP roles: [Atoti reserved roles]
            "LDAP_ATOTI_ADMIN": ["ROLE_ADMIN"],
            "LDAP_ATOTI_USER": ["ROLE_USER"],
            "LDAP_ATOTI_SHARE": ["ROLE_SHARE"],
            # LDAP roles: [user-defined roles]
            "LDAP_ROLE_INSPIRE": [ROLE_INSPIRE],
            "LDAP_ROLE_RESTAURANT": [ROLE_RESTAURANT],
            "LDAP_ROLE_BURGER": [ROLE_BURGER],
            "LDAP_ROLE_SANDWICH": [ROLE_SANDWICH],
            "LDAP_ROLE_SNACK": [ROLE_SNACK],
        }
    )

## 4. Test login and access management in web application  

Try out any of these users (Password for users created with BasicSecurity are "password"):

___Administrator___
- atoti_admin  

___Generic user___
- atoti_user   

___Inspire Brands users___
- Inspire_user1 (Access for restaurants of category Burger only)
- Inspire_user2 (Access for restaurants of category Sandwich and Snack only)
- Inspire_manager

___Restaurant Brands International LLC users___
- Restaurant_user1 (Access for restaurants of category Burger only)
- Restaurant_user2 (Access for restaurants of category Sandwich and Snack only)
- Restaurant_manager

In [30]:
session.link()

Open the notebook in JupyterLab with the Atoti extension enabled to see this link.

<div style="text-align: center;" ><a href="https://www.atoti.io/?utm_source=gallery&utm_content=security-implementation" target="_blank" rel="noopener noreferrer"><img src="https://data.atoti.io/notebooks/banners/discover-try.png" alt="Try Atoti"></a></div>